In [1]:
from core.vector_store import vector_store

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA
from langchain_g4f import G4FLLM

from g4f import Provider

/home/Artem/PycharmProjects/rag-my-base/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
retriever_model = G4FLLM(
    provider=Provider.Qwen_Qwen_2_5,
    model=Provider.Qwen_Qwen_2_5.default_model,
    create_kwargs={"temperature": 0.4}
)

retriever = vector_store.as_retriever()

retriever_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are a multilingual query rewriter helping improve document retrieval from a vector database.
    Your task is to generate several diverse reformulations of the input query to maximize recall.
    Include at least 2 queries in Russian and 2 in English. Rephrase the query using:

    - Synonyms
    - Common phrases
    - Abbreviations and their expansions
    - Related technical or domain-specific terms
    - Alternative grammatical structures

    Original user question: {question}

    Return 6–8 diverse, semantically related queries in a list format. Avoid exact duplicates. Ensure both Russian and English are represented.
    """,
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=retriever_model,
    prompt=retriever_prompt
)

In [14]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [15]:
retriever.invoke("BERT")

[Document(id='71005a36-3e1f-470e-854f-5a23c9196ca1', metadata={'arxiv_url': 'https://arxiv.org/abs/2304.054681', 'id': '2304.05468v1', 'summary': 'The Serbian language is a Slavic language spoken by over 12 million speakers and well understood by over 15 million people. In the area of natural language processing, it can be considered a low-resourced language. Also, Serbian is considered a high-inflectional language. The combination of many word inflections and low availability of language resources makes natural language processing of Serbian challenging. Nevertheless, over the past three decades, there have been a number of initiatives to develop resources and methods for natural language processing of Serbian, ranging from developing a corpus of free text from books and the internet, annotated corpora for classification and named entity recognition tasks to various methods and models performing these tasks. In this paper, we review the initiatives, resources, methods, and their avail

In [16]:
retriever_from_llm.invoke("Обратные матрицы")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Матрицы обратные', '2. Как найти обратную матрицу?', '3. Обратные матрицы в линейной алгебре', '4. Inverse matrices', '5. How to calculate the inverse of a matrix?', '6. Finding the reciprocal of a matrix', '7. Обратная матрица: методы и примеры', '8. What is an inverse matrix in linear algebra?']


[Document(id='3a8c597a-1bf7-46b3-b36f-7b6f4c2dfd3a', metadata={'title': 'Отображения', 'url': 'https://neerc.ifmo.ru/wiki/index.php?title=%D0%9E%D1%82%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F'}, page_content='Инъективноеотображение — переводит разные элементы A в разные элементы B: Сюръективноеотображение(на множестве B) — каждый элемент множества B является образом хотя бы одного элемента множества A: Биективноеотображение — инъекция + сюръекция — взаимно однозначное соответствие, обладает двумя предыдущими свойствами. - Множества'),
 Document(id='062a14c4-a970-49a3-aa47-0052ecabcf70', metadata={'title': 'Исчисление предикатов', 'url': 'https://neerc.ifmo.ru/wiki/index.php?title=%D0%98%D1%81%D1%87%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BF%D1%80%D0%B5%D0%B4%D0%B8%D0%BA%D0%B0%D1%82%D0%BE%D0%B2'}, page_content='множество[math]D[/math]- это будет множество всех существ,[math]S(x)[/math]- предикат "быть смертным",[math]H(x)[/math]- предикат "быть человеком". Тогда фраза

In [3]:
system_prompt = """You're a personal AI assistant.
Don't tell the user about the context!
Don't talk to the user about unnecessary topics other than the topic of context!
Don't provide documents unnecessarily!
Rely only on the suggested context and documents!
You should use at least one provided document to explain!
If the user asks about something else out of context, tell him "Данный вопрос находится вне базы знаний"
Along with the response, send the sources from the metadata.

Answer only on Russian, but you can think in whatever language you want.
Present all mathematical formulas in LaTeX format.

Answer the question based only on the following context:
{context}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}")
    ]
)

llm = G4FLLM(
    provider=Provider.Qwen_Qwen_2_5,
    model="qwen-qwen2-5",
    create_kwargs={"temperature": 0}
)

chain = (
    {"context": retriever_from_llm, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [4]:
user_prompt = """Объясни архитектуру GPT. В чём отличие от RNN?"""
print("AI: ", end="")
answer = ""
for chunk in chain.stream(user_prompt):
    answer += chunk
    print(chunk, end="")
print()

AI: New g4f version: 0.5.3.1 (current: 0.5.2.8) | pip install -U g4f
Архитектура GPT (Generative Pre-trained Transformer) основана на модели трансформера, которая была представлена в статье "Attention is All You Need" \cite{attentionisallyouneed}. Трансформеры используют механизм внимания (attention mechanism), который позволяет модели эффективно обрабатывать последовательности данных, особенно длинные последовательности, без потери информации.

### Архитектура GPT

1. **Механизм Внимания (Attention Mechanism)**:
   - В трансформере используется механизм внимания, который позволяет каждому элементу последовательности фокусироваться на других элементах. Это делается с помощью матриц \( Q \) (Query), \( K \) (Key) и \( V \) (Value).
   - Формула для вычисления внимания:
     \[
     \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
     \]
   - Здесь \( d_k \) — размерность ключей (keys).

2. **Многослойный Декодер (Multi-Layer Decoder)**:
   - GPT состоит 